In [4]:
branch = open("/Users/ekikodze/PycharmProjects/PlasmidTrials/1_salamin_python/branchlength.dat")
seq = open("msa.dat")
parents = open("table.dat")

In [75]:
import pandas as pd

df = pd.read_csv("table.dat", sep=',', header=None, names=["parent", "offspring"])
print(df)
branch_lengths = pd.read_csv("branchlength.dat", header=None)
branch_lengths = branch_lengths.iloc[0].tolist()
print(branch_lengths)

   parent  offspring
0       9          1
1       9          2
2       8          9
3       8          3
4       7          4
5       7          5
6       6          7
7       6          8
[0.1, 0.4, 0.01, 0.04, 0.2, 0.08, 0.12, 0.14]


In [87]:
seq = pd.read_csv("msa.dat", sep=r'\s+',header=None, names = ["node", "sequence"]  )
print(seq)

   node                        sequence
0     1  AGATCAAGATCAAGATCAAGATCAAGATCA
1     2  AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
2     3  CGCTATCGCTATCGCTATCGCTATCGCTAT
3     4  CGTTACCGTTACCGTTACCGTTACCGTTAC
4     5  CGCTACCGCTACCGCTACCGCTACCGCTAC


In [95]:
class NodeList:
    def __init__(self, df, parent_col='parent', offspring_col='offspring'):
        # Extract unique node names
        self.nodes = sorted(pd.concat([df[parent_col], df[offspring_col]]).unique().tolist())

        # Map child → parent
        self.parent_map = {row[offspring_col]: row[parent_col] for _, row in df.iterrows()}

        # Map child → branch length
        self.branch_map = {row[offspring_col]: length for (_, row), length in zip(df.iterrows(), branch_lengths)}

        # Map node -> sequence from seq dataframe if provided
        if seq is not None:
            self.seq_map = pd.Series(seq['sequence'].values, index = seq['node']).to_dict()
        else:
            self.seq_map={}

        # Build node info dictionary
        self.node_info = {}
        for node in self.nodes:
            self.node_info[node] = {
                'parent': self.parent_map.get(node),
                'branch_length': self.branch_map.get(node),
                'sequence': self.seq_map.get(node)
            }

    def show(self):
        for node, info in self.node_info.items():
            print(f"{node} ← parent: {info['parent']}, branch_length: {info['branch_length']}, sequence: {info['sequence']}")

    def to_dataframe(self):
        return pd.DataFrame([
            {'node': node,
             'parent': info['parent'],
             'branch_length': info['branch_length'],
             'sequence':info['sequence']}
            for node, info in self.node_info.items()
        ])


In [96]:
tree = NodeList(df)
tree.show()

1 ← parent: 9, branch_length: 0.1, sequence: AGATCAAGATCAAGATCAAGATCAAGATCA
2 ← parent: 9, branch_length: 0.4, sequence: AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
3 ← parent: 8, branch_length: 0.04, sequence: CGCTATCGCTATCGCTATCGCTATCGCTAT
4 ← parent: 7, branch_length: 0.2, sequence: CGTTACCGTTACCGTTACCGTTACCGTTAC
5 ← parent: 7, branch_length: 0.08, sequence: CGCTACCGCTACCGCTACCGCTACCGCTAC
6 ← parent: None, branch_length: None, sequence: None
7 ← parent: 6, branch_length: 0.12, sequence: None
8 ← parent: 6, branch_length: 0.14, sequence: None
9 ← parent: 8, branch_length: 0.01, sequence: None
